# Comparison with CaImAn
Here we run the Caiman algorithm as it is and then initializing it using the Agonia boxes or center of cells to segment and compare the extracted traces.

In [ ]:
import bokeh.plotting as bpl
import cv2
import glob
import logging
import matplotlib.pyplot as plt
import numpy as np
import os

try:
    cv2.setNumThreads(0)
except():
    pass

try:
    if __IPYTHON__:
        # this is used for debugging purposes only. allows to reload classes
        # when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
bpl.output_notebook()

import xml.etree.ElementTree as et 
import pandas as pd
from matplotlib.patches import Rectangle

import pandas as pd
import pickle
import time

## Data preparation

### Load data

Load caiman results into cnm object and Agonia boxes into ROIs

In [ ]:
#data_path = '/home/pedro/Work/AGOnIA/Boxes-data/Seeded-Caiman'
data_path = '/home/pedro/Work/AGOnIA/Boxes-data/Seeded-Caiman/prueba'
#data_name = '501271265_5000'
data_name = '501271265_5000_crop'
data_format = 'tif'
fnames = [os.path.join(data_path,data_name + '.' + data_format)]

# load boxes position info
with open(os.path.join(data_path,'501271265_boxes.pkl'),'rb') as f:
    cajas = pickle.load(f)
    f.close()
ROIs = np.empty(np.shape(np.array(cajas[:,:4]))).astype('int')
ROIs[:,[0,2]] = np.array(cajas[:,[0,2]].astype('int'))
ROIs[:,[1,3]] = np.array(cajas[:,[1,3]].astype('int'))

# load boxes temporal traces
filename = os.path.join(data_path,'patches.pkl')
with open(filename,'rb') as f:
    boxes_traces = pickle.load(f)
    f.close()

# load Caiman results
cnm = cnmf.load_CNMF(os.path.join(data_path,data_name + '_analysis_results.hdf5'))


In [ ]:
data_path = '/home/pedro/Work/AGOnIA/Boxes-data/Seeded-Caiman/prueba'
data_name = '501271265_5000_crop'
data_format = 'tif'
fnames = [os.path.join(data_path,data_name + '.' + data_format)]

# load boxes position info
with open(os.path.join(data_path,data_name + '_boxes.pkl'),'rb') as f:
    cajas = pickle.load(f)
    f.close()
ROIs = np.empty(np.shape(np.array(cajas[:,:4]))).astype('int')
ROIs[:,[0,2]] = np.array(cajas[:,[0,2]].astype('int'))
ROIs[:,[1,3]] = np.array(cajas[:,[1,3]].astype('int'))

cnm = cnmf.load_CNMF(os.path.join(data_path,data_name + '_analysis_results.hdf5'))


Data set analized with caiman without the agonia seeds 

In [ ]:
fnames = ['/home/pedro/Work/AGOnIA/Boxes-data/Caiman_test/866.avi']
cnm = cnmf.load_CNMF('/home/pedro/caiman_data/demos/notebooks/analysis_results.hdf5')
xtree = et.parse("/home/pedro/Work/AGOnIA/Boxes-data/Caiman_test/866.xml")
xroot = xtree.getroot()

df_cols = ["Cell", "xmin", "ymin", "xmax" , 'ymax']
rows = []

for i, node in enumerate(xroot[6:]): 
    box = i
    xmin = int(node.find('bndbox').find('xmin').text)
    ymin = int(node.find('bndbox').find('ymin').text)
    xmax = int(node.find('bndbox').find('xmax').text)
    ymax = int(node.find('bndbox').find('ymax').text)
    
    rows.append({"Cell": box, "xmin": xmin, 
                 "ymin": ymin, "xmax": xmax,
                 "ymax": ymax})

Box_agonia = pd.DataFrame(rows, columns = df_cols)
    

### View movies

In [ ]:
single_movie = cm.load(fnames)
print(single_movie.shape)

In [ ]:
display_movie = True
if display_movie:
    m_orig = cm.load_movie_chain(fnames)
    ds_ratio = 0.2
    m_orig.resize(1, 1, ds_ratio).play(
        q_max=99.5, fr=5, magnification=2)

### Background for boxes
Calculate the median of the recording and the sum of all the cells detected by Caiman. 
Plot AGONIA boxes on top of them. 

In [ ]:
med      = np.median(single_movie,axis=0)
max_proj = np.max(single_movie,axis=0)
caiman_cells = [np.reshape(cnm.estimates.A[:,i].toarray(), cnm.estimates.dims, order='F') for i in range(cnm.estimates.A.shape[1])]

## Plot data

### Non-seeded Caiman

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,7))
ax[0].imshow(med,cmap='gray')
for i in range(Box_agonia['Cell'].iloc[-1]):
    rect = Rectangle((Box_agonia['xmin'][i],Box_agonia['ymin'][i]), Box_agonia['xmax'][i]-Box_agonia['xmin'][i],
          Box_agonia['ymax'][i]-Box_agonia['ymin'][i],color='r',fill=False)
    ax[0].add_patch(rect)
ax[0].set_title('AGONIA over median',fontsize=15)
ax[1].imshow(np.sum(caiman_cells,axis=0))
for i in range(Box_agonia['Cell'].iloc[-1]):
    rect = Rectangle((Box_agonia['xmin'][i],Box_agonia['ymin'][i]), Box_agonia['xmax'][i]-Box_agonia['xmin'][i],
          Box_agonia['ymax'][i]-Box_agonia['ymin'][i],color='r',fill=False)
    ax[1].add_patch(rect)
ax[1].set_title('AGONIA CaImAn detection',fontsize=15)


### Seeded Caiman

Plot boxes on top of **median projection** and on top of **Caiman factors**

In [ ]:
fig,ax = plt.subplots(figsize=(12,12))
ax.imshow(med,cmap='gray')
for i in range(ROIs.shape[0]):
    rect = Rectangle((ROIs[i,0],ROIs[i,1]), ROIs[i,2]-ROIs[i,0],
          ROIs[i,3]-ROIs[i,1],color='r',fill=False)
    ax.add_patch(rect)
ax.set_title('AGONIA over median',fontsize=15)


In [ ]:
fig,ax = plt.subplots(1,2,figsize=(24,12))
ax[0].imshow(med,cmap='gray')
for i in range(ROIs.shape[0]):
    rect = Rectangle((ROIs[i,0],ROIs[i,1]), ROIs[i,2]-ROIs[i,0],
          ROIs[i,3]-ROIs[i,1],color='r',fill=False)
    ax[0].add_patch(rect)
ax[0].set_title('AGONIA over median',fontsize=15)

ax[1].imshow(np.sum(caiman_cells,axis=0))
for i in range(ROIs.shape[0]):
    rect = Rectangle((ROIs[i,0],ROIs[i,1]), ROIs[i,2]-ROIs[i,0],
          ROIs[i,3]-ROIs[i,1],color='r',fill=False)
    ax[1].add_patch(rect)
ax[1].set_title('AGONIA CaImAn detection',fontsize=15)


Use plot_contours_nb tool to find specific factors

In [ ]:
np.linspace(208,210,3)

In [ ]:
#n_cells = [185,199]
#n_cells = [19,165]
n_cells = [217]
cnm.estimates.plot_contours_nb(img=None,idx=n_cells)#idx=np.linspace(208,210,3).astype(int)# >150

Plot selected factors and the Agonia boxes

In [ ]:
fig,ax = plt.subplots(figsize=(30,20))
ax.imshow(np.sum([caiman_cells[i] for i in n_cells],axis=0),cmap='gray')
for i in n_cells:#185
    rect = Rectangle((ROIs[i,0],ROIs[i,1]), ROIs[i,2]-ROIs[i,0],
          ROIs[i,3]-ROIs[i,1],color='r',fill=False)
    ax.add_patch(rect)

View traces of selected factors acording to Caiman

In [ ]:
cnm.estimates.nb_view_components(img=med, idx=n_cells,denoised_color='red')

## Compare Agonia and Caiman temporal traces

In [ ]:
#select cells
n_cells = [185,199]
selected_cells = [np.array(boxes_traces[n]) for n in n_cells]
boxes_mean = [np.mean(selected_cells[i],axis=(0,1)) for i in range(len(n_cells))]
boxes_med = [np.median(selected_cells[i],axis=(0,1)) for i in range(len(n_cells))]
Amean_norm1 = (boxes_mean[0]-min(boxes_mean[0]))/max(boxes_mean[0]-min(boxes_mean[0]))
Amean_norm2 = (boxes_mean[1]-min(boxes_mean[1]))/max(boxes_mean[1]-min(boxes_mean[0]))

C_1 = (cnm.estimates.YrA[n_cells[0]]+cnm.estimates.C[n_cells[0]]-min(cnm.estimates.YrA[n_cells[0]]+cnm.estimates.C[n_cells[0]]))/max(cnm.estimates.YrA[n_cells[0]]+cnm.estimates.C[n_cells[0]]-min(cnm.estimates.YrA[n_cells[0]]+cnm.estimates.C[n_cells[0]]))
C_2 = (cnm.estimates.YrA[n_cells[1]]+cnm.estimates.C[n_cells[1]]-min(cnm.estimates.YrA[n_cells[1]]+cnm.estimates.C[n_cells[1]]))/max(cnm.estimates.YrA[n_cells[1]]+cnm.estimates.C[n_cells[1]]-min(cnm.estimates.YrA[n_cells[1]]+cnm.estimates.C[n_cells[1]]))

#### Plot Caiman factors on top of AGONIA mean boxes

In [ ]:
plt.figure(figsize=(20,12))
plt.subplot(211)
plt.plot(Amean_norm1)
plt.plot(C_1)
plt.subplot(212)
plt.plot(Amean_norm2)
plt.plot(C_2)


#### Correlation between Caiman factors and Agonia boxes means

In [ ]:

plt.imshow(np.corrcoef([C_1,C_2,Amean_norm1,Amean_norm2]),vmin=0,cmap='inferno')
plt.xticks([0,1,2,3],['C_1','C_2','A_mean1','A_mean2'])
plt.yticks([0,1,2,3],['C_1','C_2','A_mean1','A_mean2'])
plt.colorbar()
plt.show()

In [ ]:
plt.figure(figsize=(18,4))
plt.subplot(131)
plt.imshow(np.corrcoef([(cnm.estimates.C[n_cells[0]]-min(cnm.estimates.C[n_cells[0]]))/max(cnm.estimates.C[n_cells[0]]-min(cnm.estimates.C[n_cells[0]])),
                        (cnm.estimates.C[n_cells[1]]-min(cnm.estimates.C[n_cells[1]]))/max(cnm.estimates.C[n_cells[1]]-min(cnm.estimates.C[n_cells[1]])),
                        Amean_norm1,Amean_norm2]),vmin=0,cmap='inferno')
plt.xticks([0,1,2,3],['C_1','C_2','A_mean1','A_mean2'])
plt.yticks([0,1,2,3],['C_1','C_2','A_mean1','A_mean2'])
plt.ylim([3.5,-.5])
plt.colorbar()
plt.show()

#### Square error between Caiman factors and AGONIA mean

Selected cells

In [ ]:
plt.figure(figsize=(16,4))
dist1 = (C_1-Amean_norm1)**2
dist2 = (C_2-Amean_norm2)**2
plt.subplot(121)
h1 = plt.hist(dist1,20)
plt.subplot(122)
h2 = plt.hist(dist2,20,alpha=.5)

All cells

In [ ]:
diff = np.empty(0)
noise_diff = np.empty(0)
for i,box in enumerate(boxes_traces):
    mean_Abox = np.mean(box,axis=(0,1))
    A_trace  = (mean_Abox-min(mean_Abox))/max(mean_Abox-min(mean_Abox))
    C_trace  = (cnm.estimates.C[i]-min(cnm.estimates.C[i]))/max(cnm.estimates.C[i]-min(cnm.estimates.C[i]))
    C_wnoise =(cnm.estimates.YrA[i]+cnm.estimates.C[i]-min(cnm.estimates.YrA[i]+cnm.estimates.C[i]))/max(cnm.estimates.YrA[i]+cnm.estimates.C[i]-min(cnm.estimates.YrA[i]+cnm.estimates.C[i]))
    diff = np.append(diff,(C_trace-A_trace)**2)
    noise_diff =np.append(noise_diff,(C_wnoise-A_trace)**2)

In [ ]:
h = plt.hist(noise_diff,50)

The mean of the box is a proxy for the Caiman traces (**before denoising**) with an error of .01

In [ ]:
plt.imshow(cnm.estimates.A[:,217].toarray().reshape((512,512)))


In [ ]:
cnm.estimates.A[:,0].toarray().sum()

In [ ]:
plt.plot(mean_Abox)
plt.plot(C_trace)

In [ ]:
from sklearn.decomposition import NMF
X = np.array([selected_cells[1][:,:,i].reshape((np.size(selected_cells[1],0)*np.size(selected_cells[1],1))) for i in range(np.size(selected_cells[1],2))]).T

# fit NMF
model = NMF(n_components=2, init='random', random_state=0)
W = model.fit_transform(X)
H = model.components_


In [ ]:
plt.imshow(np.array(W[:,1]).reshape(np.shape(selected_cells[1][:,:,0])))

In [ ]:
plt.figure(figsize=(16,12))
plt.subplot(211)
plt.plot(H[0,:])
plt.subplot(212)
plt.plot(H[1,:])

In [ ]:
plt.figure(figsize=(18,6))
plt.plot(Amean_norm1)
plt.plot((cnm.estimates.C[n_cells[0]]-min(cnm.estimates.C[n_cells[0]]))/max(cnm.estimates.C[n_cells[0]]-min(cnm.estimates.C[n_cells[0]])))

In [ ]:
plt.figure(figsize=(18,6))
plt.plot(cnm.estimates.S[n_cells[0]])

In [ ]:
j=0
fig,ax = plt.subplots(figsize=(30,20))
ax.imshow(ncaiman_cells[j],cmap='gray')
for i in n_cells:#185
    rect = Rectangle((ROIs[i,0],ROIs[i,1]), ROIs[i,2]-ROIs[i,0],
          ROIs[i,3]-ROIs[i,1],color='r',fill=False)
    ax.add_patch(rect)

In [ ]:
#plt.plot(cnm.estimates.C[185])
#plt.plot(cnm.estimates.F_dff[185])
plt.plot(C_1-np.mean(C_1),alpha=.5)
plt.plot(Amean_norm1-np.mean(Amean_norm1),alpha=.5)

In [ ]:
cnm.estimates.detrend_df_f(quantileMin=8, frames_window=250)

In [ ]:
plt.plot(cnm.estimates.C[7])

In [ ]:
plt.imshow(cnm.estimates.b.reshape((512,512)).T)

In [ ]:
cnm.estimates.F_dff.shape